In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone --depth 1 https://github.com/tensorflow/models
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive
%cd /content/gdrive/MyDrive

Cloning into 'models'...
remote: Enumerating objects: 4048, done.
remote: Counting objects: 100% (4048/4048), done.
remote: Compressing objects: 100% (3076/3076), done.
remote: Total 4048 (delta 1185), reused 2794 (delta 914), pack-reused 0
Receiving objects: 100% (4048/4048), 54.69 MiB | 13.36 MiB/s, done.
Resolving deltas: 100% (1185/1185), done.


In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .

In [ ]:
!pip install pyyaml==5.3
!pip install /content/gdrive/MyDrive/models/research/
!pip install tensorflow==2.11.0

In [ ]:
import re
with open('/content/gdrive/MyDrive/models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()
with open('/content/gdrive/MyDrive/models/research/setup.py', 'w') as f:
    # Set fine_tune_checkpoint path
    s = re.sub('tf-models-official>=2.5.1',
               'tf-models-official==2.8.0', s)
    f.write(s)

In [ ]:
!python models/research/object_detection/builders/model_builder_tf2_test.py

**Create CSV data files and TFRecord files**

---



In [ ]:
!python create_csv.py
!python3 create_tfrecord.py --csv_input=images/train_labels.csv --labelmap=labelmap.txt --image_dir=images/train --output_path=train.tfrecord
!python3 create_tfrecord.py --csv_input=images/validation_labels.csv --labelmap=labelmap.txt --image_dir=images/validation --output_path=val.tfrecord

In [ ]:
train_record_fname = '/content/gdrive/MyDrive/train.tfrecord'
val_record_fname = '/content/gdrive/MyDrive/val.tfrecord'
label_map_pbtxt_fname = '/content/gdrive/MyDrive/labelmap.pbtxt'

Change the chosen_model variable to deploy different models available in the TF2 object detection zoo

In [ ]:
chosen_model = 'efficientdet-d0'

MODELS_CONFIG = {
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
    },
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
    },
    'ssd-mobilenet-v2-fpnlite-320': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
    },
}

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

In [ ]:
%mkdir /content/gdrive/MyDrive/models/mymodel/
%cd /content/gdrive/MyDrive/models/mymodel/

# Download pre-trained model weights
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

# Download training configuration file for model
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

Set training parameters for the model

In [ ]:
num_steps = 1500

if chosen_model == 'efficientdet-d0':
  batch_size = 30
else:
  batch_size = 60

Set file locations and get number of classes for config file

In [ ]:
pipeline_fname = '/content/gdrive/MyDrive/models/mymodel/' + base_pipeline_file
fine_tune_checkpoint = '/content/gdrive/MyDrive/models/mymodel/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)
print('Total classes:', num_classes)

Total classes: 1


Create custom configuration file by writing the dataset, model checkpoint, and training parameters into the base pipeline file

In [ ]:
import re

%cd /content/gdrive/MyDrive/models/mymodel
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:

    # Set fine_tune_checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # Set tfrecord files for train and test datasets
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # Set label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set batch_size
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    # Set number of classes num_classes
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    # Change fine-tune checkpoint type from "classification" to "detection"
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    # If using ssd-mobilenet-v2, reduce learning rate (because it's too high in the default config file)
    if chosen_model == 'ssd-mobilenet-v2':
      s = re.sub('learning_rate_base: .8',
                 'learning_rate_base: .08', s)

      s = re.sub('warmup_learning_rate: 0.13333',
                 'warmup_learning_rate: .026666', s)

    # If using efficientdet-d0, use fixed_shape_resizer instead of keep_aspect_ratio_resizer (because it isn't supported by TFLite)
    if chosen_model == 'efficientdet-d0':
      s = re.sub('keep_aspect_ratio_resizer', 'fixed_shape_resizer', s)
      s = re.sub('pad_to_max_dimension: true', '', s)
      s = re.sub('min_dimension', 'height', s)
      s = re.sub('max_dimension', 'width', s)

    f.write(s)


/content/gdrive/MyDrive/models/mymodel
writing custom configuration file


Set the path to the custom config file and the directory to store training checkpoints in


In [ ]:
# Set the path to the custom config file and the directory to store training checkpoints in
pipeline_file = '/content/gdrive/MyDrive/models/mymodel/pipeline_file.config'
model_dir = '/content/gdrive/MyDrive/training/'

In [ ]:
# Run training!
!python /content/gdrive/MyDrive/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1
print("Train Finish!!")

**Export inference graph**


In [ ]:
!python /content/gdrive/MyDrive/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir=/content/gdrive/MyDrive/training \
    --pipeline_config_path={pipeline_file} \
    --output_directory /content/gdrive/MyDrive/training/inferance_graph

**Loading the saved_model**


In [ ]:
%matplotlib inline
import tensorflow as tf
import time
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
from google.colab.patches import cv2_imshow
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

IMAGE_SIZE = (12, 8) # Output display size as you want
import matplotlib.pyplot as plt
PATH_TO_SAVED_MODEL= "/content/gdrive/MyDrive/training/inferance_graph/saved_model"
print('Loading model...', end='')

# Load saved model and build the detection function
detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)
print('Done!')

#Loading the label_map
category_index=label_map_util.create_category_index_from_labelmap("/content/gdrive/MyDrive/labelmap.pbtxt",use_display_name=True)

Loading model...Done!


**only the desired bounding boxes are visualized and saved**

In [ ]:
import numpy as np
import os
from PIL import Image
import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils
import matplotlib.pyplot as plt
%matplotlib inline


def load_image_into_numpy_array(path):
    return np.array(Image.open(path))

# Set the path to the directory containing your test images
IMAGE_DIR = "/content/gdrive/MyDrive/ALL_image/"

# Directory to save the bounding boxes
SAVE_DIR = "/content/gdrive/MyDrive/BB_ALL_Images/"
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

# Get all files from the directory
all_image_files = [f for f in os.listdir(IMAGE_DIR) if os.path.isfile(os.path.join(IMAGE_DIR, f))]

# Iterate over each image
for image_file in all_image_files:
    image_path = os.path.join(IMAGE_DIR, image_file)
    print(f'Running inference for {image_path}...')

    image_np = load_image_into_numpy_array(image_path)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    detections = detect_fn(input_tensor)

    # Convert batch tensors to numpy arrays and remove batch dimension
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    # Filter detections based on score and size
    valid_boxes = []
    valid_scores = []
    valid_classes = []
    for i, box in enumerate(detections['detection_boxes']):
        ymin, xmin, ymax, xmax = box
        width = (xmax - xmin) * image_np.shape[1]
        height = (ymax - ymin) * image_np.shape[0]
        if detections['detection_scores'][i] >= 0.999 and height > 51 and width > 34:
            valid_boxes.append(box)
            valid_scores.append(detections['detection_scores'][i])
            valid_classes.append(detections['detection_classes'][i])

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        np.array(valid_boxes),
        np.array(valid_classes).astype(np.int64),
        np.array(valid_scores),
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=None,
        min_score_thresh=0.999, # This is redundant now but kept for clarity
        agnostic_mode=False)

    saved_count = 0
    for i, box in enumerate(valid_boxes):
        (left, right, top, bottom) = (box[1] * image_np.shape[1], box[3] * image_np.shape[1],
                                      box[0] * image_np.shape[0], box[2] * image_np.shape[0])
        cropped_image = Image.fromarray(image_np[int(top):int(bottom), int(left):int(right)])

        # Save the bounding box as a jpg file
        cropped_image_path = os.path.join(SAVE_DIR, f"{os.path.splitext(image_file)[0]}_box_{i}.jpg")
        cropped_image.save(cropped_image_path)
        print(f'Saved box {i} to {cropped_image_path}')
        saved_count += 1

    print(f'Number of bounding boxes saved for {image_path}: {saved_count}')

    # Plot the image with detections
    plt.figure(figsize=(12, 8))
    plt.axis("off")
    plt.imshow(image_np_with_detections)
    plt.show()